In [1]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
spark = SparkSession.builder.appName('Ex2').getOrCreate()

In [6]:
dataset = spark.read.csv('./hack_data.csv',header=True,inferSchema=True)

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [9]:
feat_cols = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [10]:
vec_assembler = VectorAssembler(inputCols=feat_cols,outputCol='features')

In [11]:
final_data = vec_assembler.transform(dataset)

In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol='features',outputCol='scaledfeatures',withMean=False,withStd=True)

In [14]:
scalemodel = scaler.fit(final_data)

In [15]:
cluster_final_data = scalemodel.transform(final_data)

In [16]:
from pyspark.ml.clustering import KMeans

In [17]:
kmeans3 = KMeans(featuresCol='scaledfeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledfeatures',k=2)

In [18]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [19]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



In [20]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



Chon k=2 hop ly hon